In [1]:
!pip install selenium
!pip install webdriver

ERROR: Could not find a version that satisfies the requirement webdriver (from versions: none)
ERROR: No matching distribution found for webdriver


In [3]:
import csv 
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import os
import sys
os.path.dirname(sys.executable)
from time import sleep
from selenium import webdriver
from selenium.webdriver import Chrome 
from selenium.webdriver.chrome.options import Options as ChromeOptions
browser = webdriver.Chrome()
driver = webdriver.Chrome()

In [4]:
def get_tweet_data(card):
    """Extract data from tweet card"""
    username = card.find_element("xpath",'.//span').text
    try:
        handle = card.find_element("xpath",'.//span[contains(text(), "@")]').text
    except NoSuchElementException:
        return
    
    try:
        postdate = card.find_element("xpath",'.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    
    comment = card.find_element("xpath",'.//div[2]/div[2]/div[1]').text
    responding = card.find_element("xpath",'.//div[2]/div[2]/div[2]').text
    text = comment + responding
    reply_cnt = card.find_element("xpath",'.//div[@data-testid="reply"]').text
    retweet_cnt = card.find_element("xpath",'.//div[@data-testid="retweet"]').text
    like_cnt = card.find_element("xpath",'.//div[@data-testid="like"]').text

    
    tweet = (username, handle, postdate, text, reply_cnt, retweet_cnt, like_cnt)
    return tweet  

In [5]:
#log in to twitter
driver.get('https://twitter.com/login')

In [12]:
#skip this cell
#instead, just manually type in the search term, then navigate to 'advanced' and then type in the search 
#term and the date range. This will lead to better results


# search_term = input('search term: ')

# #driver.get('https://twitter.com/search')
# driver.maximize_window()
# sleep(5)

# # find search input and search for term
# search_input = driver.find_element("xpath",'//input[@aria-label="Search query"]')
# search_input.send_keys(search_term)
# search_input.send_keys(Keys.RETURN)
# sleep(1)

# # navigate to historical 'latest' tab
# driver.find_element_by_link_text('Latest').click()

search term: 蔣萬安


AttributeError: 'WebDriver' object has no attribute 'find_element_by_link_text'

In [6]:
#make sure you have the tab 'latest' selected before running this tap, otherwise it will only look for 
#the most relevant notes, not by date
data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling:
    page_cards = page_cards = driver.find_elements("xpath", '//article[@data-testid="tweet"]')

    #driver.find_elements_by_xpath('//article[@data-testid="tweet"]')
    for card in page_cards[-15:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
            
    scroll_attempt = 0
    while True:
        # check scroll position
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(2)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1
            
            # end of scroll region
            if scroll_attempt >= 4:
                scrolling = False
                break
            else:
                sleep(2) # attempt another scroll
        else:
            last_position = curr_position
            break

# close the web driver
driver.close()

In [7]:
data

[('翁達瑞',
  '@ong_tario',
  '2022-11-16T22:16:14.000Z',
  '翁達瑞\n@ong_tario\n·\n17 Nov 2022蔣萬安吹噓他的矽谷經驗，真的讓我看不下去。若要炫耀學經歷，這些國民黨權貴根本看不到我家的車尾燈。\n\n事實上，我曾在哈佛大學教書，我家小孩也曾在哈佛大學讀書。問題是，我行事低調，從不對外炫耀。\n\n這是第一次，詳情請看連結的貼文：\n\nhttps://facebook.com/permalink.php?story_fbid=pfbid0mtjEuWVQVkfqWtsRbjVCwcQexxg2vaW1T9uBoDvE6rfKDH6LX16D5qcL3Wuaydbil&id=100006693133292…\n37\n99\n1,315',
  '37',
  '99',
  '1,315'),
 ('Dow Chang, 稻田, 道甜, PhD&PE&MBA',
  '@dowchang',
  '2022-11-16T21:31:54.000Z',
  'Dow Chang, 稻田, 道甜, PhD&PE&MBA\n@dowchang\n·\n17 Nov 2022Replying to \n@KamiliaHaraQoo\n and \n@pinyutw2020\n蔣萬安詐騙中華民國國民, 蔣萬安沒有詐騙你國的國民!',
  '',
  '',
  ''),
 ('ChenChen',
  '@Chen57962136',
  '2022-11-16T21:13:47.000Z',
  'ChenChen\n@Chen57962136\n·\n17 Nov 2022[https://twitter.com/Chen57962136/status/1592988202096558080…]\n感想：\n藍綠造謠柯文哲臺北市政府沒做事\n但政見都在抄柯市府的\n\n像蔣萬安提的青銀共居，就是抄臺北市政府正在做的\n不是說要提政見嗎\n卻是抄柯市府？\n\n歷任市長\n只有柯文哲任內公地公有市場有改建且一堆\n柯文哲說過30分鐘內要認識城市，就去看市場[https://youtube.com/watch?v=F7vicxaKcPY…]\nChenChen\n@Chen57962136\n

In [8]:
with open('Jiang1_17.csv', 'w', newline='', encoding='utf-8') as f:
    header = ['UserName', 'Handle', 'Timestamp', 'Text', 'Comments', 'Likes', 'Retweets']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)